#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

#1 Receiving the Netlogo output
The first step is to load the files. Also, as the global variables of interest in order to associate it with the output name file.

In [13]:
import pandas as pd
import numpy as np
import copy
import random
from __future__ import division
from __future__ import with_statement
import minimization as minz #My script in same folder for minimization routines

In [34]:
#Choose here the Globals and Name used for the experiment to load.
#Make sure the files exist (i.e. from Netlogo simulations)
experiment_clue = "signalce5"#signalcesuperlong1
n_states = 8
n_signals = 1 #only without signal for now
n_rounds = 50
N = 40
n_parents = 20

#Choose generations to load ("None" to import the whole file)
start_gen=None
number_of_gens=None #So final generation imported is start_gen+number_of_gens-1

###################################################################################
###################################################################################
###################################################################################

#Equivalent of generations in the strategies file
total_pop=N*2 #total number of autos per generation
start_row_strat=(start_gen*total_pop if start_gen!=None else None)
number_rows_strat=(number_of_gens*total_pop if number_of_gens!=None else None)


globals_list = (experiment_clue, n_states, n_signals, n_rounds, N, n_parents) #Save them as a list

#Experiment name based on the chosen experiment_clue and globals
chosen_experiment = "%s_states_%s_signal_%s_rounds_%s_N_%s_parents_%s.txt" % globals_list

#Both files have to use the same "chosen experiment" (to make sure come from the same simulation in Netlogo)
summary_file_name = "summary_" + chosen_experiment #Summary output
strategies_file_name = "strategies_" + chosen_experiment #Strategies output

#Path to Netlogo outputs
netlogo_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_from_netlogo/" #Netlogo outputs in this folder

#Get the first line as header (for when importing only some generations instead of the whole file)
with open(netlogo_folder+summary_file_name, 'r') as f:
    line_s = f.readline()
    line_s = line_s.split(',')
    line_s[len(line_s)-1]=line_s[len(line_s)-1].replace('\n','')#Delete last special carachter "\n"
#For the strategies file
with open(netlogo_folder+strategies_file_name, 'r') as f:
    line_st = f.readline()
    line_st = line_st.split(',')
    line_st[len(line_st)-1]=line_st[len(line_st)-1].replace('\n','')
    
#Read files and save them as data
df_sum = pd.read_csv(netlogo_folder + summary_file_name,\
                     skiprows=start_gen,nrows=number_of_gens)

df_strat = pd.read_csv(netlogo_folder + strategies_file_name,\
                       skiprows=start_row_strat,nrows=number_rows_strat)
#Replace header using first row of file (instead of from imported data)
df_sum.columns=line_s
df_strat.columns=line_st

summary = df_sum.copy() #allows to go directly to regime characterization by average (without joint machines)

#2 Minimise the automata
Use the functions to have a simple code here for minimising the auto and storing other relevant variables (as available states, etc)

In [52]:
#Other required globals
n_obs = 2 if n_signals == 0 else 4 #Define here (or change) the possible observations of the machines

canon_autos_list = [] #Create empty lists to store the processed autos below
min_autos_list = []


for i in df_strat.index: #For each row...
    if i%1000 == 0:
        print 'machine ',i, ' out of ', len(df_strat.index)
    netlogo_auto = df_strat.auto_long[i] #... for all netlogo_auto
    big_auto = minz.to_format_netlogo_auto(netlogo_auto) #Use function to convert the raw Netlogo auto in a list format
    init_state = big_auto[0] #Save initial state of the machine
    normal_auto = minz.new_empty_auto(n_obs, n_states) #Use function to create a new empty auto as a numpy array

    # Next block it to fill the new 'normal_auto' with the information from big_auto.
    # The objective is that normal_auto=big_auto but as an array (instead of a list)
    my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
    for i, j in enumerate(my_index):
        normal_auto['actions'][i] = big_auto[j]
        normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        
    canon_auto = minz.convert_to_canonical(normal_auto, n_states, init_state, n_obs) #Use function for canonical form
    
    access_states = len(canon_auto) #n_states now is only the accesible states of the machine (before minimization)
    
    #Use function to get minimum behavioural equivalent auto
    #Passes "0" as 3rd argument because that's init_state now (always 0 for canonical auto)
    min_auto = minz.minimized_automaton(canon_auto, access_states, 0, n_obs)
    
    #Update autos lists
    canon_autos_list.append(canon_auto) #Store proccessed autos in the corresponding list
    min_autos_list.append(min_auto)
    
#Add the processed autos lists as columns to df_strat
df_strat["canon_autos"] = canon_autos_list #Add the lists with autos to the dataframe
df_strat["min_autos"] = min_autos_list

machine  0  out of  160000
machine  1000  out of  160000
machine  2000  out of  160000
machine  3000  out of  160000
machine  4000  out of  160000
machine  5000  out of  160000
machine  6000  out of  160000
machine  7000  out of  160000
machine  8000  out of  160000
machine  9000  out of  160000
machine  10000  out of  160000
machine  11000  out of  160000
machine  12000  out of  160000
machine  13000  out of  160000
machine  14000  out of  160000
machine  15000  out of  160000
machine  16000  out of  160000
machine  17000  out of  160000
machine  18000  out of  160000
machine  19000  out of  160000
machine  20000  out of  160000
machine  21000  out of  160000
machine  22000  out of  160000
machine  23000  out of  160000
machine  24000  out of  160000
machine  25000  out of  160000
machine  26000  out of  160000
machine  27000  out of  160000
machine  28000  out of  160000
machine  29000  out of  160000
machine  30000  out of  160000
machine  31000  out of  160000
machine  32000  out o

#3 Accesible states and minimum behavioural states

In [53]:
access_states = [len(x) for x in df_strat.canon_autos] #accesible states in the big machine
min_states = [len(x) for x in df_strat.min_autos] #accesible states in the minimized machine

df_strat["access_states"] = access_states
df_strat["min_states"] = min_states

#4 Joint machines (not minimized)

In [54]:
# Lists to keep track of joint machines
print 'start'
gen_list = []
jm_list = []
parents_index_list = []
for gen in df_sum.generation: #each generation
    #if gen%10 == 0:
    print "generation ", gen, ' out of ', len(df_sum.generation) #Useful for debugging (or keeping track of simulation time)
    df_col = df_strat[(df_strat.population == "column") & (df_strat.generation == gen)][:] #column autos for this gen
    df_row = df_strat[(df_strat.population == "row") & (df_strat.generation == gen)][:] #row autos for this gen
    
    for i0, auto0 in enumerate(df_col.min_autos):#minimized autos in population col, for this gen
        index_0=df_col.index[i0]
        
        for i1, auto1 in enumerate(df_row.min_autos):#minimized autos in population row, for this gen
            index_1=df_row.index[i1]
            
            if n_signals==1: #With signal
                jm = minz.create_joint_machine_with_signal(auto0, auto1) #Function to create the joint machine
            if n_signals==0: #No signal
                jm = minz.create_joint_machine_no_signal(auto0, auto1) #Function to create the joint machine
            
            gen_list.append(gen) #Keeps track of generation
            jm_list.append(jm)   #Keeps track of joint machines
            #Parents_index_list Keeps track of joint machine constituent machines' location (index) in df_strat
            parents_index_list.append([index_0, index_1]) #Needed for Unused states measure
            
#Store the joint machines, generation and parent index (in a new dataframe)
df_jms = pd.DataFrame(columns = ("generation", "jm", "parents_index")) #Store joint machines with associated generation
df_jms.generation = gen_list
df_jms.jm = jm_list
df_jms.parents_index = parents_index_list

start
generation  70000  out of  2000
generation  70001  out of  2000
generation  70002  out of  2000
generation  70003  out of  2000
generation  70004  out of  2000
generation  70005  out of  2000
generation  70006  out of  2000
generation  70007  out of  2000
generation  70008  out of  2000
generation  70009  out of  2000
generation  70010  out of  2000
generation  70011  out of  2000
generation  70012  out of  2000
generation  70013  out of  2000
generation  70014  out of  2000
generation  70015  out of  2000
generation  70016  out of  2000
generation  70017  out of  2000
generation  70018  out of  2000
generation  70019  out of  2000
generation  70020  out of  2000
generation  70021  out of  2000
generation  70022  out of  2000
generation  70023  out of  2000
generation  70024  out of  2000
generation  70025  out of  2000
generation  70026  out of  2000
generation  70027  out of  2000
generation  70028  out of  2000
generation  70029  out of  2000
generation  70030  out of  2000
ge

#5 Minimize the joint machines 
Adds the 'min_jm' column to df_jms (tuples with the minimized joint machines)

In [55]:
min_jm_list = [] #to save the minimized joint machines, and add later to the dataframe (df_jms)

for i, jm in enumerate(df_jms.jm): #all joint machines
    if i%50000 == 0:
        print "joint machine ", i, ' out of ', len(df_jms.jm)
#for jm in [df_jms.jm[24000], df_jms.jm[24001]]:

    if n_signals==1:#with signal
        canon_jm = minz.convert_to_canonical(jm, len(jm), 0, 2)
        min_jm = minz.minimized_automaton(canon_jm, len(canon_jm), 0, 2)

        #Next lines convert the min_jms, which is a dict, into a tuple
        #Converts an structured numpy array ("actions" and 'transitions' in the min_jm)into a tuple of tuples
        #This is so that it can be used as a key to use groupby (since tuple is inmutable)
        tup_transitions = tuple(tuple(pair_transitions) for pair_transitions in min_jm["transitions"])
        tup_actions = tuple(min_jm['actions'])
        min_jm = tuple(zip(tup_actions,tup_transitions))
    
    if n_signals==0:#no signal
        min_jm = minz.minimize_joint_machine_no_signal(jm) #minimize them (have actions and cyclestart)
        #Next lines convert the min_jms, which is a dict, into a tuple
        #Converts a list of lists ("actions" in the min_jm)into a tuple of tuples
        #This is so that it can be used as a key to use groupby (since tuple is inmutable)
        tup_actions = tuple(tuple(pair_actions) for pair_actions in min_jm["actions"]) #convert actions to tuples
        min_jm = (tup_actions, min_jm["cyclestart"]) #add the cyclestart to final min_jm tuple        
    

    min_jm_list.append(min_jm) #save the minimized machine to a list

# Save the minimized joint machines
df_jms["min_jms"] = None #new empty column in dataframe
df_jms.min_jms = min_jm_list #add the minimized joint machines to the dataframe

joint machine  0  out of  3200000
joint machine  50000  out of  3200000
joint machine  100000  out of  3200000
joint machine  150000  out of  3200000
joint machine  200000  out of  3200000
joint machine  250000  out of  3200000
joint machine  300000  out of  3200000
joint machine  350000  out of  3200000
joint machine  400000  out of  3200000
joint machine  450000  out of  3200000
joint machine  500000  out of  3200000
joint machine  550000  out of  3200000
joint machine  600000  out of  3200000
joint machine  650000  out of  3200000
joint machine  700000  out of  3200000
joint machine  750000  out of  3200000
joint machine  800000  out of  3200000
joint machine  850000  out of  3200000
joint machine  900000  out of  3200000
joint machine  950000  out of  3200000
joint machine  1000000  out of  3200000
joint machine  1050000  out of  3200000
joint machine  1100000  out of  3200000
joint machine  1150000  out of  3200000
joint machine  1200000  out of  3200000
joint machine  1250000  ou

#Probability Density Function for not minimized joint machines (with signal)

In [56]:
#if n_signals == 0:
#    pass

if n_signals == 1:
    pdf_list = [] #to save the pdf of each joint machine and add later to the dataframe (df_jms)
    
    for i,jm in enumerate(df_jms.jm):
        if i%50000 == 0:
            print "joint machine ", i, ' out of ', len(df_jms.jm)
        #Here jm is the joint machine (not minimized), followed by the number of signals to feed the machine with,
        #followed by the number of trials (repeats of feeding the signal). The total number of repetitions to obtain
        #the PDF, is then tt*trials. Check the function in 'minimization'.
        pdf = minz.prob_density_function_joint_machine_with_signal(jm,100,100)#arguments are auto, tt, trials
        pdf_list.append(pdf)
    
    # Save the minimized joint machines
    df_jms["pdf_long_jm"] = None #new empty column in dataframe
    df_jms.pdf_long_jm = pdf_list #add the minimized joint machines to the dataframe

#Used states

In [57]:
#Prepare dataframe to keep track of unused states
df_strat["used_states"] = 0 # Will contain a list with states of each min_auto
used_states_list = []
for i, auto in enumerate(df_strat.min_autos): #all minimized autos
    a = [0 for ix in xrange(df_strat.min_states[i])] #List the size of minimised machine's states
    used_states_list.append(a)
df_strat["used_states"] = used_states_list #Add to dataframe


if n_signals==0: #No signal
    
    for i,jm in enumerate(df_jms.jm): #all joint machines. They are not minimized.
        if i%50000 == 0:
            print "joint machine ", i, ' out of ', len(df_jms.jm)
        index_0 = df_jms['parents_index'][i][0]#Index reference for parent machine (parents are in df_strat)
        index_1 = df_jms['parents_index'][i][1]
    
    
        for st in jm["states"]: #for metastates in jointmachine
            s0 = st[0] #state that is used by parent0 (stored in current metastate)
            s1 = st[1]
            #Go to the parent machine (in df_strat) and alter the 'used_states' list
            df_strat["used_states"][index_0][s0] = 1# =1 for states visited. Unvisited remain 0
            df_strat["used_states"][index_1][s1] = 1

if n_signals==1: #with signal
    
    for i,jm in enumerate(df_jms.jm): #all joint machines. They are not minimized.
        if i%1600 == 0:
            print "joint machine ", i, ' out of ', len(df_jms.jm)
        #The key here is that in the joint machines (in position 2 of each metastate) there is the information of
        #the states used by each component machine. For example, if the joint machine's metastate 3 is formed by states
        # 4 and 0 of parent0 and parent1, respectively, then jm[3][2]=[4,0]. So this information is contanined in 
        #the list 's', accesed when marking the used states (given by a positive probability in the pdf).
        s = [x[2] for x in jm]
        
        index_0 = df_jms['parents_index'][i][0]#Index reference for parent machine (parents are in df_strat)
        index_1 = df_jms['parents_index'][i][1]
        #print 'jm index = ',i
        #print 'index_0 = ',index_0
        #print 'index_1 = ',index_1
        
        pdf = df_jms.pdf_long_jm[i] #pdf of the joint machine
        used_jm_states_1 = [i if x[1]>0 else None for i,x in enumerate(pdf)] #has positions of used metastates (positive probability in pdf)
        used_jm_states = [x for x in used_jm_states_1[:] if x!=None]#remove all "None" entries
        #print 'used_jm_states',used_jm_states
        
        for st in used_jm_states: #all metastates with positive probability
            parent0_state = s[st][0] #state used by parent0 in current used metastate
            #print 'parent0_state = ',parent0_state
            df_strat.used_states[index_0][parent0_state] = 1 #mark parent0's used states list (1=used, 0=unused)
            #print 'parent0 used_state = ',df_strat.used_states[index_0]          
            
            #print 'current st = ',st
            parent1_state = s[st][1]
            #print 'parent1_state = ',parent1_state
            df_strat.used_states[index_1][parent1_state] = 1
            #print 'parent1 used_state = ',df_strat.used_states[index_1]    
        #print "\n"


joint machine  0  out of  3200000
joint machine  50000  out of  3200000
joint machine  100000  out of  3200000
joint machine  150000  out of  3200000
joint machine  200000  out of  3200000
joint machine  250000  out of  3200000
joint machine  300000  out of  3200000
joint machine  350000  out of  3200000
joint machine  400000  out of  3200000
joint machine  450000  out of  3200000
joint machine  500000  out of  3200000
joint machine  550000  out of  3200000
joint machine  600000  out of  3200000
joint machine  650000  out of  3200000
joint machine  700000  out of  3200000
joint machine  750000  out of  3200000
joint machine  800000  out of  3200000
joint machine  850000  out of  3200000
joint machine  900000  out of  3200000
joint machine  950000  out of  3200000
joint machine  1000000  out of  3200000
joint machine  1050000  out of  3200000
joint machine  1100000  out of  3200000
joint machine  1150000  out of  3200000
joint machine  1200000  out of  3200000
joint machine  1250000  ou

#6 Frequencies of minimized joint machines
Outputs dataframe "freqjm" with frequencies of joint machines
(Does it by transforming df_jms)

In [58]:
#Use Groupby and organize the data set for frequencies

g1 = df_jms.copy() #use intermediate copies to avoid potential bugs later. Not sure if actually needed...
g1 = g1.groupby([g1["generation"], g1["min_jms"]]) #split by groups
g1 = g1.count() #organize as frequency of joint machine per generation

interactions = N * N #number of joint machines per generation
freq_perc_list = [(x*100)/interactions for x in g1.jm] #list with frequency percentage of jm per generation
g1['freq_perc'] = freq_perc_list #add frequency to the dataframe

#jm_freq_threshold = 0 #Change to higher for easier visualization
#g1 = g1[g1.freq_perc > jm_freq_threshold] #keep machines with frequency higher than threshold


#Organise the dataframe

freqjm = g1.copy() #just in case...
freqjm = freqjm.rename(columns = {'jm':'freq'}) #rename column
freqjm = freqjm.reset_index() #reset_index converts the multiindex into normal columns (to use generation for 'sort')
freqjm = freqjm.sort(['generation', 'freq_perc'], ascending=[True, False]) #sort


#If no signal, show the lollipop machine as a string. Example: "AA BB >>AA<<"" for a machines that plays first
#AA, then BB, and then forever plays AA (whatever is inside >> << is the metamachine cycle)
if n_signals==0:#no signal
    jm_show = [minz.min_jm_no_signal_to_string(x) for x in freqjm.min_jms] #use function to convert to string
    freqjm['jm_show'] = jm_show #add to dataframe

#With signal, perhaps the jm_show (a good way to show the joint machine), is by using the Markov matrix
if n_signals==1:#no signal
    pass

#7 Unused behaviour and slack in construction measures

Unused not ready for signal. Check later

In [59]:
#Unused states: number of states not visited in the minimized machine
unused_states = [len(x) - x.count(1) for x in df_strat.used_states] #unused states in min_autos

#Unvisited: potential for novel behavior given change in the  input stream. Is unused states divided by min_states
unvisited_measure = [(len(x)-x.count(1))/len(x) for x in df_strat.used_states]

#Behaviour_slack: slack in the potential behavior of the machine
#the more states you use, the more sophisticated you can become behaviorally.
behaviour_slack = [len(x)/n_states for x in df_strat.min_autos] #min_lenght/total states.

#construction_slack: slack in the construction of the complete machine
construction_slack = [x/n_states for x in df_strat.access_states]#accesible/total

df_strat['unused_states'] = unused_states
df_strat['unvisited_measure'] = unvisited_measure    
df_strat['behaviour_slack'] = behaviour_slack
df_strat['construction_slack'] = construction_slack

#df_strat = df_strat.drop('used_states', 1)

In [60]:
#Take the average per generation of unused states, unvisited measure, and slack measures

strats = df_strat.copy() #just in case
strats = strats.groupby(strats.generation).mean() #take the mean of all the variables (by generation)
strats = strats.drop(['ID','score',],1) #not needed (1 is to drop columns instead of rows)
strats = strats.reset_index()

In [61]:
#Take average of same measures, but per population

strats_col = df_strat.copy()
strats_col = strats_col[strats_col.population == 'column'] #choose only one population

strats_row = df_strat.copy()
strats_row = strats_row[strats_row.population == 'row']

def change_columns_names(df, to_add): #changes the names of the columns of the datafrae, to add, for example, "_row"
    old_names = df.columns
    #change all names, expect 'generation'
    new_names = [name+'%s'%to_add if name!='generation' else name for name in old_names[:]]
    df.columns = new_names
    return df

strats_row = change_columns_names(strats_row, '_row')#use function to change column names
strats_col = change_columns_names(strats_col, '_col')

strats_row = strats_row.groupby(strats_row.generation).mean()#take mean of variables (per generation, per population)
strats_col = strats_col.groupby(strats_col.generation).mean()

strats_row = strats_row.reset_index() #organise index
strats_col = strats_col.reset_index()

to_delete_row = ['ID_row','score_row',' ce_individual_row']#drop some variables
to_delete_col = ['ID_col','score_col',' ce_individual_col']#drop some variables
strats_row = strats_row.drop(to_delete_row,1)
strats_col = strats_col.drop(to_delete_col,1)

In [62]:
#Organise summary dataframe, and include the measures calculated above.

summary = df_sum.copy() #just in case
#delete columns that won't use
#to_delete = ['row_heads_A', 'row_heads_B', 'row_tails_A', 'row_tails_B', 'col_heads_A', 'col_heads_B',\
#'col_tails_A','col_tails_B','times_heads','times_tails']
#summary = summary.drop(to_delete, axis=1)

summary = pd.merge(summary, strats, on='generation') #merge datasets
summary = pd.merge(summary, strats_row, on='generation')
summary = pd.merge(summary, strats_col, on='generation')

#8 Regime identification
Two regime classifications: based on top joint machine and based on percentage of play

In [63]:
#By top joint machine:
#Function to find the highest frequency percentage top machine
def find_top_jm (df, n=1, column='freq_perc'):
    return df.sort_index(by=column)[-n:]

#Apply the function to get the highest frequency joint machine per generation
topjm = freqjm.groupby('generation').apply(find_top_jm)

#Define regime as the top joint machine in a generation if its frequency is above the defined "regime_threshold"
#percentage. If none is above it, the regime is in "other"
regime_threshold = 50 
regime_jm = [jm if int(topjm.freq_perc[i]) > regime_threshold else 'not_threshold' for i,jm in enumerate(topjm.min_jms)]

#Add regime to summary dataframe
summary['regime_jm'] = regime_jm

In [35]:
#By percentage:
regime_av = [None for i in summary.index] #Create variable to fill

#Regime based on percetanges of machines playing AA or BB
for i in summary.index: #all generations
    
    A = summary.coordination_A_perc[i] #percentage of AA plays for this generation
    B = summary.coordination_B_perc[i] #percentage of BB plays for this generation
    
    if A > 0.8:
        regime_av[i] = 'Domination_AA'
    elif B > 0.8:
        regime_av[i] = 'Domination_BB'
    elif A > 0.35 and A < 0.55 and B > 0.35 and B < 0.55:
        regime_av[i] = 'Turn_Taking'
    elif A > 0.5 and A < 0.8 and B > 0.2 and B < 0.5:
        regime_av[i] = 'Biased_Turn_A'
    elif B > 0.5 and B < 0.8 and A > 0.2 and A < 0.5:
        regime_av[i] = 'Biased_Turn_B'
    else:
        regime_av[i] = 'Other'
        
summary['regime_av'] = regime_av  
#summary

#9 Epoch characterization
Two Epoch matrices are constructed: "epoch_av", with epochs based on the average percentage of AA and BB per generation (computationally much faster and doesn't require previous procedures but only the summary matrix), and "epoch_jm" which is based on the top machine of the generation

An epoch is defined as having the top joint machine on a generation (i.e. regime) to be the same over a window of 
past generations (e.g. was this period the same regime as in the past ones?). In that window of past regimes, some tolerance is permitted (i.e. some of them can be different, allowing for some errors).

The algorithm considers an epoch to have started when a regime appears a minimum number of times in the window of past regimes.

In [36]:
#Epoch characterization works well with 10 and 3 for long simulations
epoch_window = 10 #lagged regimes to be considered
epoch_tolerance = 3 #number of misses in the window before breaking an epoch

#Use custom function to generate epoch matrix
epochs_av=minz.epoch_matrix(summary,'regime_av',epoch_window,epoch_tolerance) #based on average percentages of (AA,BB)
#print epochs_av

In [66]:
#Requires Joint Machines

epochs_jm=minz.epoch_matrix(summary,'regime_jm',epoch_window,epoch_tolerance) #based on top joint machine
#print epochs_jm

#10 Export main dataframes
Export the three main data frames, so I can work with graphs and statistics from a different file. This makes the scripts a bit more modular, and also I just have to run the minimization procedures (this file) only once per experiment.

In [37]:
output_file_modifier='' #To change the name of output file

#Path to python outputs
python_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_from_python/" #Netlogo outputs in this folder

In [38]:
#Based on percentage of plays (AA and BB)
epochs_av.to_csv((python_folder+'epochs_av_'+output_file_modifier+chosen_experiment))

#summary: contains main variables. Averages per generation
summary.to_csv(python_folder+'summary_'+output_file_modifier+chosen_experiment)

In [69]:
#freqjm: frequency of each joint machine per generation.
freqjm.to_csv(python_folder+'jm_'+output_file_modifier+chosen_experiment)

#epochs: each row has the regime 
#Top joint machine (if above threshold, otherwise "not_threshold")
epochs_jm.to_csv((python_folder+'epochs_jm_'+output_file_modifier+chosen_experiment))

#STOP! no need to run the script further
###Just for visualization
freqjm: frequency of all joint machines per generation (for transition analysis)

summary: main variables (averages per generation)

epochs: classification of epochs, with duration, starting and ending periods.

In [70]:
#For visualization of joint machines with different frequency thresholds

jm_freq_threshold = 50 #Change to higher for easier visualization (percentage)
freqjm[freqjm.freq_perc > jm_freq_threshold] #keep machines with frequency higher than threshold


,generation,min_jms,freq,parents_index,freq_perc,jm_show
29,70000,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1147,1147,71.6875,AA BB >> AA AA BB AA <<
53,70001,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1147,1147,71.6875,AA BB >> AA AA BB AA <<
87,70002,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1332,1332,83.2500,AA BB >> AA AA BB AA <<
107,70003,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1190,1190,74.3750,AA BB >> AA AA BB AA <<
134,70004,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1330,1330,83.1250,AA BB >> AA AA BB AA <<
158,70005,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1224,1224,76.5000,AA BB >> AA AA BB AA <<
200,70006,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1085,1085,67.8125,AA BB >> AA AA BB AA <<
234,70007,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1221,1221,76.3125,AA BB >> AA AA BB AA <<
283,70008,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1156,1156,72.2500,AA BB >> AA AA BB AA <<
311,70009,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",1225,1225,76.5625,AA BB >> AA AA BB AA <<


In [71]:
summary

,generation,av_score_row,av_score_col,miscoordination_perc,coordination_B_perc,coordination_A_perc,row_heads_A,row_heads_B,row_tails_A,row_tails_B,...,behaviour_slack_row,construction_slack_row,access_states_col,min_states_col,unused_states_col,unvisited_measure_col,behaviour_slack_col,construction_slack_col,regime_jm,regime_av
0,70000,2.159875,2.562937,0.055438,0.270750,0.673813,0,0,0,0,...,0.862500,0.865625,7.975,7.925,0.000,0.000000,0.990625,0.996875,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
1,70001,2.203575,2.570425,0.045200,0.293975,0.660825,0,0,0,0,...,0.859375,0.871875,8.000,8.000,0.000,0.000000,1.000000,1.000000,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
2,70002,2.241175,2.587638,0.034237,0.309650,0.656112,0,0,0,0,...,0.853125,0.865625,7.950,7.925,0.925,0.116071,0.990625,0.993750,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
3,70003,2.221812,2.583938,0.038850,0.299513,0.661637,0,0,0,0,...,0.856250,0.875000,8.000,8.000,0.000,0.000000,1.000000,1.000000,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
4,70004,2.230488,2.568388,0.040225,0.310937,0.648837,0,0,0,0,...,0.884375,0.884375,7.975,7.925,0.025,0.003125,0.990625,0.996875,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
5,70005,2.173662,2.533525,0.058563,0.290787,0.650650,0,0,0,0,...,0.875000,0.875000,7.975,7.975,0.025,0.003125,0.996875,0.996875,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
6,70006,2.186575,2.551862,0.052312,0.291200,0.656488,0,0,0,0,...,0.875000,0.875000,7.925,7.900,0.050,0.006250,0.987500,0.990625,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
7,70007,2.057500,2.417562,0.104987,0.267475,0.627537,0,0,0,0,...,0.878125,0.878125,8.000,7.950,0.025,0.003125,0.993750,1.000000,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
8,70008,2.145150,2.505288,0.069913,0.284975,0.645112,0,0,0,0,...,0.868750,0.871875,8.000,7.950,0.075,0.009821,0.993750,1.000000,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A
9,70009,2.231238,2.595325,0.034688,0.300613,0.664700,0,0,0,0,...,0.859375,0.865625,7.975,7.925,0.000,0.000000,0.990625,0.996875,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",Biased_Turn_A


In [72]:
epochs_av

,epoch,duration,start,end
1,Biased_Turn_A,1990,10,2000


In [73]:
epochs_jm

,epoch,duration,start,end
1,"(((A, A), (B, B), (A, A), (A, A), (B, B), (A, ...",394,10,404
2,not_threshold,38,417,455
3,not_threshold,3,469,472
4,"(((A, A), (B, A), (A, A), (A, A), (A, B), (A, ...",14,477,491
5,not_threshold,49,498,547
6,not_threshold,23,565,588
7,"(((A, B), (A, A), (A, A), (B, B), (A, A)), 1)",82,595,677
8,not_threshold,39,684,723
9,"(((A, A), (B, A), (A, A), (A, A), (A, B), (A, ...",4,729,733
10,not_threshold,6,743,749


In [74]:
df = freqjm[freqjm.freq_perc > jm_freq_threshold]
jm_list = df.min_jms

auto = jm_list[48]

pdf = minz.prob_density_function_joint_machine_with_signal(auto,1000,1000)#arguments are auto, tt, trials

print pdf

KeyError: 48